# ground truth for foundpose

generate ground truth pose from COLMAP reconstructed camera poses + manually found world pose

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
from pathlib import Path
from typing import List

import cv2
import numpy as np
from PIL import Image
import plotly.graph_objects as go
import pyrender
from tqdm import tqdm
import transforms3d as t3d
import trimesh
import visu3d as v3d
import yaml

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_models, to_contour
from burybarrel.camera import load_v3dcams
from burybarrel.utils import add_to_json
from burybarrel.plotting import get_surface_line_traces, get_ray_trace

In [ ]:
name = "dive8-barrel-14-32"
datadir = Path(f"/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/data/input_data/{name}")
resdir = Path(f"/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results/{name}")
camposes_path = resdir / "colmap-out/cam_poses.json"

infopath = Path("../configs/blender_gt_info.yaml")
with open(infopath, "rt") as f:
    allinfo = yaml.safe_load(f)
info = allinfo[name]
model_path = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/models3d") / info["object_name"]
cams, imgpaths = load_v3dcams(camposes_path, img_parent=datadir / "rgb")
scalefactor = info["scalefactor"]

def T_from_blender(Reuler, t, scalefactor):
    R = t3d.euler.euler2mat(*(np.array(Reuler) * np.pi / 180))
    t = np.array(t) * (1 / scalefactor)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    T = v3d.Transform.from_matrix(T)
    return T
T_gt = T_from_blender(info["R"], info["t"], scalefactor)
T_floor_gt = T_from_blender(info["R_floor"], info["t_floor"], scalefactor)
imgs = np.array([cv2.cvtColor(cv2.imread(imgpath), cv2.COLOR_BGR2RGB) for imgpath in imgpaths])

In [ ]:
mesh: trimesh.Trimesh = trimesh.load(model_path)
meshvol = mesh.volume
vtxs = np.array(mesh.vertices)
rgb = np.zeros_like(vtxs, dtype=np.uint8)
rgb[:, 0] = 255
vtxs_p3d = v3d.Point3d(p=vtxs, rgb=rgb)

In [ ]:
# scaled 3d
camscaled = cams.replace(world_from_cam=cams.world_from_cam.replace(t=cams.world_from_cam.t * (1 / scalefactor)))
gtoverlaydir = datadir / "gt-overlays"
gtoverlaydir.mkdir(exist_ok=True)
gttmp = gtoverlaydir / "tmp"
gttmp.mkdir(exist_ok=True)
# visualization of GT
plane = trimesh.creation.box(extents=(10, 10, 0.01))
for i, img in enumerate(tqdm(imgs)):
    imgpath = imgpaths[i]
    vtxs_trf = T_gt @ vtxs_p3d
    rgb, _, _ = render_models(camscaled[i], mesh, T_gt, light_intensity=200.0)
    # Image.fromarray(rgb).save(gttmp / f"{imgpath.stem}.png")
    overlayimg = to_contour(rgb, color=(255, 0, 0), background=img)
    # these won't actually be used, just for visual reference, jpg so it's smaller
    Image.fromarray(overlayimg).save(gtoverlaydir / f"{imgpath.stem}.jpg")
    # Image.fromarray(render_v3d(camscaled[i], vtxs_trf, radius=4, background=img)).save(gtoverlaydir / f"{imgpaths[i].stem}.png")
    rgb_primitives, _, _ = render_models(camscaled[i], [mesh, plane], [T_gt, T_floor_gt], light_intensity=200.0, flags=pyrender.RenderFlags.NONE)
    Image.fromarray(rgb_primitives).save(gtoverlaydir / f"{imgpath.stem}_primitives.jpg")

In [ ]:
# idk
floornorm = T_floor_gt.apply_to_dir(np.array([0, 0, 1]))
zup_mesh_T = T_floor_gt.inv @ T_gt
zup_mesh = mesh.copy().apply_transform(zup_mesh_T.matrix4x4)
mesh_zvals = zup_mesh.vertices[:, 2]
zmin, zmax = np.min(mesh_zvals), np.max(mesh_zvals)
if zmin >= 0:
    burial_ratio_z = 0
else:
    burial_ratio_z = abs(zmin) / (abs(zmin) + zmax)
slicedmesh = trimesh.intersections.slice_mesh_plane(zup_mesh, [0, 0, 1], [0, 0, 0], cap=True)
burial_ratio_vol = 1 - slicedmesh.volume / meshvol
print(f"vol burial: {burial_ratio_vol}, z level burial: {burial_ratio_z}")
xx, yy = np.meshgrid(np.linspace(-0.2, 0.2, 10), np.linspace(-0.2, 0.2, 10))
zz = np.zeros_like(xx)
raycent = np.mean(zup_mesh.vertices, axis=0)
plane = go.Surface(x=xx, y=yy, z=zz, opacity=0.2)
v3d.make_fig(v3d.Point3d(p=zup_mesh_T @ mesh.vertices), plane, *get_surface_line_traces(xx, yy, zz), get_ray_trace(raycent, [0, 0, 1], color="#ff4d00", length=0.1, width=5, markersize=10))

In [ ]:
obj2cams_truth = [cam.world_from_cam.inv @ T_gt for cam in camscaled]
floor2cams_truth = [cam.world_from_cam.inv @ T_floor_gt for cam in camscaled]
gt_data_list = []
for i, (T, T_floor) in enumerate(zip(obj2cams_truth, floor2cams_truth)):
    truthdata = {
        "img_path": str(imgpaths[i]),
        "R": T.R.tolist(),
        "t": T.t.tolist(),
        "R_floor": T_floor.R.tolist(),
        "t_floor": T_floor.t.tolist(),
    }
    gt_data_list.append(truthdata)
with open(datadir / "gt_obj2cam.json", "wt") as f:
    json.dump(gt_data_list, f, indent=4)
add_to_json({"burial_ratio_vol": burial_ratio_vol, "burial_ratio_z": burial_ratio_z}, datadir / "info.json")